In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# %cd '/content/drive/MyDrive/Colab Notebooks/DeepFake Detector'
%cd '..'

In [ ]:
!pip install face_recognition

In [ ]:
import os
from tqdm import tqdm
from PIL import Image
from heapq import nsmallest
from math import ceil
from time import time
from random import randint, seed
from pipeline import *
seed(0)

In [ ]:
real_video_dir_path = './Celeb-DF-v2/Celeb-real-videos'
fake_video_dir_path = './Celeb-DF-v2/Celeb-synthesis-videos'

In [ ]:
# video_dir_path = real_video_dir_path
video_dir_path = fake_video_dir_path
video_listdir = os.listdir(video_dir_path)

In [ ]:
# 1 pic per vid
for video_name in tqdm(video_listdir):
    start_time = time()
    
    video = os.path.join(video_dir_path, video_name)
    frames = get_every_frame(video, interval=1)
    
    # face_coordinates = get_face_locations(frames)
    face_coordinates = get_face_locations(frames, GPU=True, batch_size=96)
    centroid = get_centroid(face_coordinates)
    dist_from_centroid = get_distance_from_centroid(centroid,
                                                    face_coordinates)

    movement_thershold = nsmallest(4, set([d for d in dist_from_centroid if d is not None]))[-1]
    

    stable_faces = get_stable_faces(movement_thershold,
                                dist_from_centroid, 
                                face_coordinates, 
                                frames, 
                                zoomed=False)

    number_of_frames_w_faces = len(stable_faces)
    sections = 6
    differencing_interval = ceil(number_of_frames_w_faces/sections)
    video_name = video_name[:-4]

    differenced_face = difference(stable_faces, interval=differencing_interval)
    Image.fromarray(differenced_face, mode='RGB').save(
        os.path.join(f'{video_dir_path}-diff', f'{video_name}.png'),'PNG')

    average_face = average(stable_faces)
    Image.fromarray(average_face, mode='RGB').save(
        os.path.join(f'{video_dir_path}-avg', f'{video_name}.png'),'PNG')

    random_face = stable_faces[randint(0, number_of_frames_w_faces-1)]
    Image.fromarray(random_face, mode='RGB').save(
        os.path.join(f'{video_dir_path}-rnd', f'{video_name}.png'),'PNG')

    print(f'\n{video_name} pre-processed in {time()-start_time:.2f}s')

In [ ]:
# N pic per vid
for video_name in tqdm(video_listdir):
    start_time = time()
    
    video = os.path.join(video_dir_path, video_name)
    frames = get_every_frame(video, interval=1)
    
    # face_coordinates = get_face_locations(frames)
    face_coordinates = get_face_locations(frames, GPU=True, batch_size=96)
    centroid = get_centroid(face_coordinates)
    dist_from_centroid = get_distance_from_centroid(centroid,
                                                    face_coordinates)

    movement_thershold = nsmallest(4, set([d for d in dist_from_centroid if d is not None]))[-1]
    

    stable_faces = get_stable_faces(movement_thershold,
                                dist_from_centroid, 
                                face_coordinates, 
                                frames, 
                                zoomed=False)

    number_of_frames_w_faces = len(stable_faces)
    sections = 6
    differencing_interval = ceil(number_of_frames_w_faces/sections)
    video_name = video_name[:-4]

    differenced_face = difference_all(stable_faces, interval=differencing_interval)
    Image.fromarray(differenced_face, mode='RGB').save(
        os.path.join(f'{video_dir_path}-diff', f'{video_name}.png'),'PNG')

    average_face = average_all(stable_faces)
    Image.fromarray(average_face, mode='RGB').save(
        os.path.join(f'{video_dir_path}-avg', f'{video_name}.png'),'PNG')

    
    random_face = stable_faces[randint(0, number_of_frames_w_faces-1)]
    Image.fromarray(random_face, mode='RGB').save(
        os.path.join(f'{video_dir_path}-rnd', f'{video_name}.png'),'PNG')

    print(f'\n{video_name} pre-processed in {time()-start_time:.2f}s')